In [ ]:
import cv2
import os
from ultralytics import YOLO

# Load your fine-tuned YOLOv8 model
model = YOLO('your-model-path')

# Define the input videos and output paths
video_paths = {
    'scene-index': 'your-input-video-path',
}

output_dir = 'your-output-video-path'
os.makedirs(output_dir, exist_ok=True)

# Define IoU function
def calculate_iou(box1, box2):
    x1, y1, x2, y2 = box1
    x1_b, y1_b, x2_b, y2_b = box2

    xi1, yi1 = max(x1, x1_b), max(y1, y1_b)
    xi2, yi2 = min(x2, x2_b), min(y2, y2_b)
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)

    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2_b - x1_b) * (y2_b - y1_b)
    union_area = box1_area + box2_area - inter_area

    iou = inter_area / union_area if union_area > 0 else 0
    return iou

# Function to perform object tracking and save the output video with counters
def track_objects_in_video(video_path, output_video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video {video_path}")
        return

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
    video_writer = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), frame_rate, (frame_width, frame_height))

    tracks = []  # Initialize object tracks
    next_track_id = 1  # Initialize the next track ID

    # Object counter variables
    class_labels = {0: 'Vehicle', 1: 'Male', 2: 'Female'}  # Map class IDs to labels

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Reset object counter for this frame
        frame_counter = {'Vehicle': 0, 'Male': 0, 'Female': 0}

        # Perform object detection using YOLOv8 model
        results = model(frame)  # Run inference on the frame

        # Extract the bounding boxes, class IDs, and confidence scores
        detections = []
        for box in results[0].boxes:  # Iterate through detected boxes
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls = int(box.cls[0])
            conf = box.conf[0]
            detections.append({'bbox': [x1, y1, x2, y2], 'class': cls, 'conf': conf})

        # Sort detections by confidence score in descending order
        detections = sorted(detections, key=lambda x: x['conf'], reverse=True)

        # Keep track of which tracks have been assigned in this frame
        assigned_tracks = set()

        updated_tracks = []

        for detection in detections:
            class_id = detection['class']
            class_name = class_labels.get(class_id, 'Unknown')

            # Update the frame-specific object counter
            if class_name in frame_counter:
                frame_counter[class_name] += 1

            best_iou = 0
            best_track_idx = -1
            for i, track in enumerate(tracks):
                if i in assigned_tracks:
                    continue  # Skip already assigned tracks

                iou = calculate_iou(detection['bbox'], track['bbox'])
                if iou > best_iou and iou > 0.3:  # IoU threshold
                    best_iou = iou
                    best_track_idx = i

            if best_track_idx >= 0:
                # Update existing track
                tracks[best_track_idx]['bbox'] = detection['bbox']
                updated_tracks.append(tracks[best_track_idx])
                assigned_tracks.add(best_track_idx)
            else:
                # Create a new track
                new_track = {'id': next_track_id, 'bbox': detection['bbox'], 'class': detection['class']}
                updated_tracks.append(new_track)
                next_track_id += 1

        # Replace old tracks with updated tracks
        tracks = updated_tracks

        # Draw bounding boxes and track IDs on the frame
        for track in tracks:
            x1, y1, x2, y2 = track['bbox']
            class_name = class_labels.get(track['class'], 'Unknown')
            # Draw the object class and ID on the bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f'ID: {track["id"]} {class_name}', (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Display object counter on the video
        counter_text = f'Vehicles: {frame_counter["Vehicle"]} | Males: {frame_counter["Male"]} | Females: {frame_counter["Female"]}'
        cv2.putText(frame, counter_text, (10, frame_height - 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # Write the processed frame to the output video
        video_writer.write(frame)

    # Release resources
    cap.release()
    video_writer.release()
    print(f"Tracking video saved at {output_video_path}")

# Run object tracking on all videos with object counter
for bg_num, video_path in video_paths.items():
    output_video_path = os.path.join(output_dir, f'tracked_background_{bg_num}_with_counter.mp4')
    track_objects_in_video(video_path, output_video_path)